In [1]:
import openpyxl as opl
#import related to tkinter file dialog
from tkinter import *
from tkinter import ttk
from tkinter import filedialog as fd
import pandas as pd
import os
import math
import datetime
import shutil
import numpy as np
#logg;ing related info
import logging
import xlsxwriter
import pandas as pd
from scipy import interpolate
import numpy as np
import math

In [2]:
"""This Files Saves a list of Data Frame"""
def saveDataFrame(dataframes,filepath,names):
    #writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    #writer= pd.ExcelWriter(filepath)
    with pd.ExcelWriter(filepath) as writer:
        for df,sname in zip(dataframes,names):        
            df.to_excel(writer,sheet_name=sname) 
    
    #writer.save()  
"""This Files Writes Data Frame in Proper Place of a Excel Sheet"""
"""Functions for Transferring Data Frame to Autocad Input File"""
def writeDataFrameToExcel(data_frame,drow,dcol,wbook_name,wsheet_name):
    """
    Parameters:
    data_frame= name of the frame from which data to be transferred
    drow=row offset to be added to dframe row index
    dcol=column offset to be added to dframe column index
    wbook_name=name of the destination work book
    wsheet_name=name of the destination work sheet    
    """
    
    wb=opl.load_workbook(wbook_name)
    #print(wb.sheetnames)
    transfer_sheet=wb[wsheet_name]
    dfshape=data_frame.shape 
    print(dfshape)
    for i in range(0,dfshape[0]):
        row_index=i+drow
        for j in range(0,dfshape[1]):
            if np.isscalar(data_frame.iloc[i,j]):                
                transfer_sheet.cell(row=row_index,column=j+dcol).value=data_frame.iloc[i,j]
            else:
                print(data_frame.iloc[i,j])
                transfer_sheet.cell(row=row_index,column=j+dcol).value=float(data_frame.iloc[i,j])

    wb.save(wbook_name)

In [3]:
"""This Function Checks Length of Arrays in Dictionary.Which is necessary for Pandas Data Frame Creation """
def check_array_length_for_df(data_dict):
    for key,values in data_dict.items():
        print("list {} has lenngth={}".format(key,len(values)))

In [4]:
"""This function calculate_return_period_correction_factor """
def calculate_return_period_correction_factor(T,rt_df):
    """
    parameter description:
    T: return period for which correction shall be calcualted
    rt_df:table of correction    
    """
    x=rt_df['Return_Period']
    y=rt_df['Multyplying _Factor']
    f1=interpolate.interp1d(x,y)
    mf=f1(T)
    return mf
    pass

In [5]:
"""This function calculate Basin Characteristics Correction Factor"""
def calculate_basin_characteristics_correction_factor(sf,adjustment_df):
    """
        parameter description:
        sf: [(L**2)/s]^0.3
        rt_df:table of correction    
    """
    #Slope_Factor	Adjustment
    if sf>7:
        cf=-40
    else:        
        x=adjustment_df['Slope_Factor']
        y=adjustment_df['Adjustment']
        f2=interpolate.interp1d(x,y)
        cf=f2(sf)
    return cf   

In [6]:
"""This function calculate Basin Size Correction Factors"""
def calculate_basin_size_correction_factor(basin_area):
    if basin_area>40:
        cf=20
    else:
        cf=70-1.25*basin_area
    return cf
    pass


In [7]:
"""This Function Converts MKS input data to FPS """
def covert_main_df_to_fps(input_df):
    output_df=input_df.copy(deep=True)
    output_df["Basin_Area"]=output_df["Basin_Area"]/(1.609*1.609)
    #Length of Khal	Slope
    output_df["Length_of_Khal"]=output_df["Length_of_Khal"]/(1.609)
    output_df["Slope"]=output_df["Slope"]/(1000*100)
    return output_df

In [8]:
"""This class solves uniform flow problem for trapozoidal section"""
class Trapozoidal_Xsec:    
    def __init__(self,Q,B,m,s_0,n):
        self.Q=Q
        self.B=B
        self.m=m
        self.s_0=s_0
        self.n=n
        print("n={} type={}".format(n,type(n)))
        print("Q={} type={}".format(Q,type(Q)))
              
        self.SF=(n*Q)/math.sqrt(s_0)
    def calculate_section_params(self,h):
        Q=self.Q
        B=self.B
        m=self.m
        s_0=self.s_0
        n=self.n
        SF=self.SF
        A=B*h+m*h*h
        dp_dh=2*math.sqrt(1+m*m)
        P=dp_dh*h+B
        R=A/P
        T=B+2*m*h
        R_2_3=math.pow(R,(2.0/3.0))
        R_5_3=math.pow(R,(5.0/3.0))
        f_h=A*R_2_3-SF
        f_prime_h=R_2_3*T+R_5_3*dp_dh
        h_calc=h-(f_h/f_prime_h)
        Q_calc=(1.0/n)*A*R_2_3*math.sqrt(s_0)
        myvalues={"h":h,"Q_design":Q,"n":n,"s_0":s_0,"B":B,"m":m,
                  "A":A,"P":P,"R":R,"T":T,"dp_dh":dp_dh,"R_2_3":R_2_3,"R_5_3":R_5_3,
                  "f_h":f_h,"f_prime_h":f_prime_h,"h_calc":h_calc,"Q_calc":Q_calc}
        return myvalues
        pass
    def solve_for_h(self,h_initial):
        initial_values={ "trial_no":[],"h":[],"Q_design":[],"n":[],"s_0":[],"B":[],"m":[],
            "A":[],"P":[],"R":[],"T":[],"dp_dh":[],"R_2_3":[],"R_5_3":[],
                  "f_h":[],"f_prime_h":[],"h_calc":[],"Q_calc":[]}
        result_df=pd.DataFrame(initial_values)
        #df1=pd.DataFrame(initial_values)
        tol=h_initial
        h0=h_initial
        h1=h_initial
        iter_no=1
        while tol>0.001 and iter_no<50:
            myvalues=self.calculate_section_params(h0)
            dict1={"trial_no":int(iter_no)}
            dict1.update(myvalues)
            new_row = pd.Series(dict1)
            result_df.loc[len(result_df)]=new_row
            #df2=pd.DataFrame(myvalues)
            #df1=df1.append(df2)
            logger.debug(dict1)
            h1=myvalues["h_calc"]
            tol=h0-h1
            h0=h1
            iter_no+=1
        return result_df
        pass
        
    def display(self):
        logger.debug("\n\n\n########Displaying Trapozoidal Sections Propertied################\n")
        Q=self.Q
        B=self.B
        m=self.m
        s_0=self.s_0
        n=self.n
        SF=self.SF
        logger.debug("Q={} B={} m={} s_0={} n={} SF={}".format(Q,B,m,s_0,n,SF))

In [9]:
"""
#############################################################################################################################
Programme Execution Begins Here 
#############################################################################################################################

"""
#create an instance of tkinter frame or windowroot=Tk()
#create an instance of tkinter frame or window
root=Tk()
root.geometry("750x150")
work_book_path=fd.askopenfilename(title="select Excel Input File")
root.withdraw()
#print(work_book_path)
dir_name=os.path.dirname(work_book_path)
head,tail=os.path.split(work_book_path)
print(dir_name)
print(tail)
file_names=tail.split(".")
print(dir_name)
out_path=os.path.join(dir_name,"Hydrologic_Design_"+file_names[0]+"."+file_names[1])
out_path2=os.path.join(dir_name,"data_agg_output.xlsx")

D:/Design_All_DC_5/AEMostofa/Southwest Project/SMO Gopalgonj/Khal/P1/XL DATA
Hydrologic Design Input.xlsx
D:/Design_All_DC_5/AEMostofa/Southwest Project/SMO Gopalgonj/Khal/P1/XL DATA


In [10]:
#inthis cell wefind direction of datamatrix

In [11]:
#settingpup log files
log_file_path=os.path.join(dir_name,"data_agg.log")
logger=logging.getLogger(__name__)
logger.setLevel("DEBUG")
fileHandler=logging.FileHandler(filename=log_file_path,mode="w")
formatter=logging.Formatter('%(asctime)s - %(name)s - %(lineno)s - %(levelname)s - %(message)s')
fileHandler.setFormatter(formatter)
fileHandler.setLevel(logging.DEBUG)
logger.addHandler(fileHandler)
#formatter=logging.Formatter("{asctime}-{levelname}-{message}", style="{",datefmt="%Y-%m-%d %H:%M %S")
#testing logger Setup........................
logger.debug("Sucessfully Setup Logger for this programme.............")
#creating a second log file path and handler for handling individual function
log_file_path_function=os.path.join(dir_name,"function_log.log")
fileHandler_function=logging.FileHandler(filename=log_file_path_function,mode="w")
fileHandler_function.setFormatter(formatter)
fileHandler_function.setLevel(logging.DEBUG)

                                    

In [12]:
#work_book_path
shheetName="Main_Input"
#sheet_name
Main_Input_Df_Mks=pd.read_excel(work_book_path,sheet_name=shheetName)
Main_Input_Df_Mks

,Code_Name,Khal_Name,Basin_Area,Length_of_Khal,Slope,Return_Period,q_100,Bed _Width,side_slope_m,Mannings_n
0,HBK,Hiron Branch Khal,2,0.678,5,10,155,3,2,0.03
1,HMP,Hiron Middle Para Khal,2,0.714,5,10,155,3,2,0.03
2,BHK,Boroi Hati Khal,2,0.600,5,10,155,3,2,0.03
3,RKK,Rakhila Khal,2,1.768,5,10,155,3,2,0.03
4,GZK,Gazir Khal,3,2.325,5,10,155,3,2,0.03
5,SSK,Silna -Shir Khal,2,1.570,5,10,155,3,2,0.03
6,GBK,Gabir Beel Khal,3,2.170,5,10,155,3,2,0.03
7,KPK,Khanar Par Khal,3,2.330,5,10,155,3,2,0.03
8,DLB,Dola Beel,3,2.260,5,10,155,3,2,0.03
9,DLK,Dhulorujir Khal,2,0.950,5,10,155,3,2,0.03


In [13]:
shheetName="Return_Period_Correction"
Return_Period_Correction_Df=pd.read_excel(work_book_path,sheet_name=shheetName)
Return_Period_Correction_Df

,Return_Period,Multyplying _Factor
0,500,1.14
1,200,1.06
2,50,0.93
3,25,0.85
4,15,0.79
5,10,0.73
6,5,0.62
7,2,0.40


In [14]:
shheetName="Basin_Slope_Correction"
Basin_Slope_Correction_Df=pd.read_excel(work_book_path,sheet_name=shheetName)
Basin_Slope_Correction_Df

,Slope_Factor,Adjustment
0,0.98,40
1,2.00,10
2,3.00,-10
3,4.00,-22
4,5.00,-31
5,6.00,-38
6,7.00,40


In [15]:
Main_Input_Df_fps=covert_main_df_to_fps(Main_Input_Df_Mks)
Main_Input_Df_fps

,Code_Name,Khal_Name,Basin_Area,Length_of_Khal,Slope,Return_Period,q_100,Bed _Width,side_slope_m,Mannings_n
0,HBK,Hiron Branch Khal,0.772535,0.421380,0.00005,10,155,3,2,0.03
1,HMP,Hiron Middle Para Khal,0.772535,0.443754,0.00005,10,155,3,2,0.03
2,BHK,Boroi Hati Khal,0.772535,0.372902,0.00005,10,155,3,2,0.03
3,RKK,Rakhila Khal,0.772535,1.098819,0.00005,10,155,3,2,0.03
4,GZK,Gazir Khal,1.158802,1.444997,0.00005,10,155,3,2,0.03
5,SSK,Silna -Shir Khal,0.772535,0.975761,0.00005,10,155,3,2,0.03
6,GBK,Gabir Beel Khal,1.158802,1.348664,0.00005,10,155,3,2,0.03
7,KPK,Khanar Par Khal,1.158802,1.448104,0.00005,10,155,3,2,0.03
8,DLB,Dola Beel,1.158802,1.404599,0.00005,10,155,3,2,0.03
9,DLK,Dhulorujir Khal,0.772535,0.590429,0.00005,10,155,3,2,0.03


In [16]:
mf=calculate_return_period_correction_factor(17,Return_Period_Correction_Df)
print(mf)

0.802


In [17]:
khal_name=Main_Input_Df_fps.iloc[11,1]
A=Main_Input_Df_fps.iloc[11,2]
L=Main_Input_Df_fps.iloc[11,3]
s=Main_Input_Df_fps.iloc[11,4]
T=Main_Input_Df_fps.iloc[11,5]
q_100=Main_Input_Df_fps.iloc[11,6]
print("khal name={} slope={} T={} q_100={}".format(khal_name,s,T,q_100))
mf=calculate_return_period_correction_factor(T,Return_Period_Correction_Df)
q_10=q_100*mf
slope_factor=math.pow((L**2)/s,0.3)
print("khal name={} q_100={} mf={} q_10={}".format(khal_name,q_100,mf,q_10))
print("slope factor={}".format(slope_factor))
##############################
cf=calculate_basin_characteristics_correction_factor(slope_factor,Basin_Slope_Correction_Df)
q_adjusted=q_10*(1+cf/100.0)
print("Khal name={} sf={} correction={} q_adjusted={}".format(khal_name,slope_factor,cf,q_adjusted))
bcf=calculate_basin_size_correction_factor(A)
q_corrected=q_adjusted*(1+bcf/100.0)
print("Khal name={} A={} bcf={} q_adjusted={} q_corrected={}".format(khal_name,A,bcf,q_adjusted,q_corrected))

khal name=Kochu Khal slope=5e-05 T=10 q_100=155
khal name=Kochu Khal q_100=155 mf=0.73 q_10=113.14999999999999
slope factor=24.240555065359917
Khal name=Kochu Khal sf=24.240555065359917 correction=-40 q_adjusted=67.88999999999999
Khal name=Kochu Khal A=1.1588018143746277 bcf=68.55149773203172 q_adjusted=67.88999999999999 q_corrected=114.4296118102763


In [18]:
khal_name=Main_Input_Df_fps.iloc[12,1]
A=Main_Input_Df_fps.iloc[12,2]
L=Main_Input_Df_fps.iloc[12,3]
s=Main_Input_Df_fps.iloc[12,4]
T=Main_Input_Df_fps.iloc[12,5]
q_100=Main_Input_Df_fps.iloc[12,6]
print("khal name={} slope={} T={} q_100={}".format(khal_name,s,T,q_100))
mf=calculate_return_period_correction_factor(T,Return_Period_Correction_Df)
q_10=q_100*mf
slope_factor=math.pow((L**2)/s,0.3)
print("khal name={} q_100={} mf={} q_10={}".format(khal_name,q_100,mf,q_10))
print("slope factor={}".format(slope_factor))
##############################
cf=calculate_basin_characteristics_correction_factor(slope_factor,Basin_Slope_Correction_Df)
q_adjusted=q_10*(1+cf/100.0)
print("Khal name={} sf={} correction={} q_adjusted={}".format(khal_name,slope_factor,cf,q_adjusted))
bcf=calculate_basin_size_correction_factor(A)
q_corrected=q_adjusted*(1+bcf/100.0)
print("Khal name={} A={} bcf={} q_adjusted={} q_corrected={}".format(khal_name,A,bcf,q_adjusted,q_corrected))

khal name=Aria Khal slope=5e-05 T=10 q_100=155
khal name=Aria Khal q_100=155 mf=0.73 q_10=113.14999999999999
slope factor=10.84997149054973
Khal name=Aria Khal sf=10.84997149054973 correction=-40 q_adjusted=67.88999999999999
Khal name=Aria Khal A=0.3862672714582092 bcf=69.51716591067724 q_adjusted=67.88999999999999 q_corrected=115.08520393675876


In [19]:
 Main_Input_Df_fps

,Code_Name,Khal_Name,Basin_Area,Length_of_Khal,Slope,Return_Period,q_100,Bed _Width,side_slope_m,Mannings_n
0,HBK,Hiron Branch Khal,0.772535,0.421380,0.00005,10,155,3,2,0.03
1,HMP,Hiron Middle Para Khal,0.772535,0.443754,0.00005,10,155,3,2,0.03
2,BHK,Boroi Hati Khal,0.772535,0.372902,0.00005,10,155,3,2,0.03
3,RKK,Rakhila Khal,0.772535,1.098819,0.00005,10,155,3,2,0.03
4,GZK,Gazir Khal,1.158802,1.444997,0.00005,10,155,3,2,0.03
5,SSK,Silna -Shir Khal,0.772535,0.975761,0.00005,10,155,3,2,0.03
6,GBK,Gabir Beel Khal,1.158802,1.348664,0.00005,10,155,3,2,0.03
7,KPK,Khanar Par Khal,1.158802,1.448104,0.00005,10,155,3,2,0.03
8,DLB,Dola Beel,1.158802,1.404599,0.00005,10,155,3,2,0.03
9,DLK,Dhulorujir Khal,0.772535,0.590429,0.00005,10,155,3,2,0.03


In [20]:
khal_name_list=[]
A_list=[]
L_list=[]
s_list=[]
T_list=[]
q_100_list=[]
mf_list=[]
q_10_list=[]
slope_factor_list=[]
cf_list=[]
q_adjusted_list=[]
bcf_list=[]
q_corrected_list=[]
Q_cusec_list=[]
Q_cumec_list=[]
Code_Name_list=[]
B_list=[]
m_list=[]
n_list=[]

for index,row in Main_Input_Df_fps.iterrows():
    Code_Name=Main_Input_Df_fps.iloc[index,0]
    Code_Name_list.append(Code_Name)
    khal_name=Main_Input_Df_fps.iloc[index,1]    
    A=Main_Input_Df_fps.iloc[index,2]
    L=Main_Input_Df_fps.iloc[index,3]
    s=Main_Input_Df_fps.iloc[index,4]
    T=Main_Input_Df_fps.iloc[index,5]
    q_100=Main_Input_Df_fps.iloc[index,6]
    B=Main_Input_Df_fps.iloc[index,7]
    m=Main_Input_Df_fps.iloc[index,8]
    n=Main_Input_Df_fps.iloc[index,9]
    #Code_Name	Khal_Name	Basin_Area	Length_of_Khal	Slope	Return Period	q_100
    khal_name_list.append(khal_name)
    A_list.append(A)
    L_list.append(L)
    s_list.append(s)
    T_list.append(T)
    q_100_list.append(q_100)
    B_list.append(B)
    m_list.append(m)
    n_list.append(n)
    """ 
    khal_name=row['Khal_Name'].iloc[0]    
    A=row['Basin_Area'].iloc
    L=Main_Input_Df_fps['Length_of_Khal']
    s=row['Slope']
    T=row['Return_Period']
    q_100=row['q_100'] 
    """
    print("khal name={} slope={} T={} q_100={}".format(khal_name,s,T,q_100))
    mf=calculate_return_period_correction_factor(T,Return_Period_Correction_Df)
    mf_list.append(mf)
    q_10=q_100*mf
    q_10_list.append(q_10)
    slope_factor=math.pow((L**2)/s,0.3)
    slope_factor_list.append(slope_factor)
    print("khal name={} q_100={} mf={} q_10={}".format(khal_name,q_100,mf,q_10))
    print("slope factor={}".format(slope_factor))
    ##############################
    cf=calculate_basin_characteristics_correction_factor(slope_factor,Basin_Slope_Correction_Df)
    cf_list.append(cf)
    q_adjusted=q_10*(1+cf/100.0)
    q_adjusted_list.append(q_adjusted)
    print("Khal name={} sf={} correction={} q_adjusted={}".format(khal_name,slope_factor,cf,q_adjusted))
    bcf=calculate_basin_size_correction_factor(A)
    bcf_list.append(bcf)
    q_corrected=q_adjusted*(1+bcf/100.0)
    q_corrected_list.append(q_corrected)
    Q_cusec= q_corrected*A
    Q_cumec=Q_cusec/35.314
    Q_cusec_list.append( Q_cusec)
    Q_cumec_list.append(Q_cumec)
    print("Khal name={} A={} bcf={} q_adjusted={} q_corrected={}".format(khal_name,A,bcf,q_adjusted,q_corrected))
    print("Khal name={} A={} q_corected={} Q_cusec={} Q_cumec={}".format(khal_name,A,q_corrected,Q_cusec,Q_cumec))
    pass


khal name=Hiron Branch Khal slope=5e-05 T=10 q_100=155
khal name=Hiron Branch Khal q_100=155 mf=0.73 q_10=113.14999999999999
slope factor=11.617513573344834
Khal name=Hiron Branch Khal sf=11.617513573344834 correction=-40 q_adjusted=67.88999999999999
Khal name=Hiron Branch Khal A=0.7725345429164184 bcf=69.03433182135447 q_adjusted=67.88999999999999 q_corrected=114.75740787351754
Khal name=Hiron Branch Khal A=0.7725345429164184 q_corected=114.75740787351754 Q_cusec=88.65406163784087 Q_cumec=2.510450859088205
khal name=Hiron Middle Para Khal slope=5e-05 T=10 q_100=155
khal name=Hiron Middle Para Khal q_100=155 mf=0.73 q_10=113.14999999999999
slope factor=11.983793016467931
Khal name=Hiron Middle Para Khal sf=11.983793016467931 correction=-40 q_adjusted=67.88999999999999
Khal name=Hiron Middle Para Khal A=0.7725345429164184 bcf=69.03433182135447 q_adjusted=67.88999999999999 q_corrected=114.75740787351754
Khal name=Hiron Middle Para Khal A=0.7725345429164184 q_corected=114.75740787351754 Q

In [21]:
""" 
khal_name_list=[]
A_list=[]
L_list=[]
s_list=[]
T_list=[]
q_100_list=[]
mf_list=[]
q_10_list=[]
slope_factor_list=[]
cf_list=[]
q_adjusted_list=[]
bcf_list=[]
q_corrected_list=[]
Q_cusec_list=[]
Q_cumec_list=[]
"""
#Code_Name	Khal_Name	Basin_Area	Length_of_Khal	Slope	Return_Period	q_100

data={ "Code_Name":Code_Name_list,
     "Khal_Name":khal_name_list,
      "Basin_Area":A_list,
      "Length_of_Khal":L_list,
      "Slope":s_list,
      "Return_Period":T_list,
      "q_100":q_100_list,
      "return_period_adjustment" :mf_list,
      "q_10":q_10_list,
      "Basin_Charcteristic_factor":slope_factor_list,
      "Basin_Charcteristic_Correction":cf_list,
      "q_10_adjusted":q_adjusted_list,
      "Basin_Area_Correction":bcf_list,
      "q_10_corrected":q_corrected_list,
      "Q_cusec":Q_cusec_list,
      "Q_cumec":Q_cumec_list, 
      "B":B_list,
      "m":m_list,
      "n":n_list,
      
     }

check_array_length_for_df(data)
Hydrologic_Analysis_Df=pd.DataFrame(data)

list Code_Name has lenngth=19
list Khal_Name has lenngth=19
list Basin_Area has lenngth=19
list Length_of_Khal has lenngth=19
list Slope has lenngth=19
list Return_Period has lenngth=19
list q_100 has lenngth=19
list return_period_adjustment has lenngth=19
list q_10 has lenngth=19
list Basin_Charcteristic_factor has lenngth=19
list Basin_Charcteristic_Correction has lenngth=19
list q_10_adjusted has lenngth=19
list Basin_Area_Correction has lenngth=19
list q_10_corrected has lenngth=19
list Q_cusec has lenngth=19
list Q_cumec has lenngth=19
list B has lenngth=19
list m has lenngth=19
list n has lenngth=19


In [22]:
    """
    Parameters:
    data_frame= name of the frame from which data to be transferred
    drow=row offset to be added to dframe row index
    dcol=column offset to be added to dframe column index
    wbook_name=name of the destination work book
    wsheet_name=name of the destination work sheet    
    """

fname_combined_input_all=os.path.join(dir_name,"Hydraulogic_Design_output.xlsx")
shutil.copy("Blank_Ouput_Hydrologic_Design.xlsx",fname_combined_input_all)

drow=1
dcol=1
wsheet_name="Design_Q"
wbook_name=fname_combined_input_all
data_frame=Hydrologic_Analysis_Df
writeDataFrameToExcel(data_frame,2,1,fname_combined_input_all,wsheet_name)
#writeDataFrameToExcel(data_frame,drow,dcol,wbook_name,wsheet_name)



(19, 19)
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.73


In [23]:
Hydrologic_Analysis_Df

,Code_Name,Khal_Name,Basin_Area,Length_of_Khal,Slope,Return_Period,q_100,return_period_adjustment,q_10,Basin_Charcteristic_factor,Basin_Charcteristic_Correction,q_10_adjusted,Basin_Area_Correction,q_10_corrected,Q_cusec,Q_cumec,B,m,n
0,HBK,Hiron Branch Khal,0.772535,0.421380,0.00005,10,155,0.73,113.15,11.617514,-40,67.89,69.034332,114.757408,88.654062,2.510451,3,2,0.03
1,HMP,Hiron Middle Para Khal,0.772535,0.443754,0.00005,10,155,0.73,113.15,11.983793,-40,67.89,69.034332,114.757408,88.654062,2.510451,3,2,0.03
2,BHK,Boroi Hati Khal,0.772535,0.372902,0.00005,10,155,0.73,113.15,10.796081,-40,67.89,69.034332,114.757408,88.654062,2.510451,3,2,0.03
3,RKK,Rakhila Khal,0.772535,1.098819,0.00005,10,155,0.73,113.15,20.647369,-40,67.89,69.034332,114.757408,88.654062,2.510451,3,2,0.03
4,GZK,Gazir Khal,1.158802,1.444997,0.00005,10,155,0.73,113.15,24.334877,-40,67.89,68.551498,114.429612,132.601242,3.754920,3,2,0.03
5,SSK,Silna -Shir Khal,0.772535,0.975761,0.00005,10,155,0.73,113.15,19.227161,-40,67.89,69.034332,114.757408,88.654062,2.510451,3,2,0.03
6,GBK,Gabir Beel Khal,1.158802,1.348664,0.00005,10,155,0.73,113.15,23.348082,-40,67.89,68.551498,114.429612,132.601242,3.754920,3,2,0.03
7,KPK,Khanar Par Khal,1.158802,1.448104,0.00005,10,155,0.73,113.15,24.366263,-40,67.89,68.551498,114.429612,132.601242,3.754920,3,2,0.03
8,DLB,Dola Beel,1.158802,1.404599,0.00005,10,155,0.73,113.15,23.924366,-40,67.89,68.551498,114.429612,132.601242,3.754920,3,2,0.03
9,DLK,Dhulorujir Khal,0.772535,0.590429,0.00005,10,155,0.73,113.15,14.223600,-40,67.89,69.034332,114.757408,88.654062,2.510451,3,2,0.03


In [24]:
"""
Code_Name 	Khal_Name 	Basin_Area 	Length_of_Khal 	Slope 	
Return_Period 	q_100 	return_period_adjustment 	q_10 	Basin_Charcteristic_factor 	
Basin_Charcteristic_Correction 	q_10_adjusted 	
Basin_Area_Correction 	q_10_corrected 	
Q_cusec 	Q_cumec 	B 	m 	n
"""
mynames=[]
myframes=[]

calcualtion_path=os.path.join(dir_name,"Section_Design_Trials.xlsx")
for index,row in Hydrologic_Analysis_Df.iterrows():
    code=row["Code_Name"]
    khal=row["Khal_Name"]
    s_0=row["Slope"]
    Q=row["Q_cumec"]
    B=row["B"]
    m=row["m"]
    n=row["n"]
    mysection=Trapozoidal_Xsec(Q,B,m,s_0,n)
    print("Code={}   khal={}".format(code,khal))
    h=20.0
    solution_df=mysection.solve_for_h(h)
    myframes.append(solution_df.copy(deep=True))
    mynames.append(khal)
    solution_df=None
    #mysection.display()
print("Total dataframe={} total names={}".format(len(myframes),len(mynames)))
saveDataFrame(myframes,calcualtion_path,mynames)

n=0.03 type=<class 'float'>
Q=2.510450859088205 type=<class 'float'>
Code=HBK   khal=Hiron Branch Khal
n=0.03 type=<class 'float'>
Q=2.510450859088205 type=<class 'float'>
Code=HMP   khal=Hiron Middle Para Khal
n=0.03 type=<class 'float'>
Q=2.510450859088205 type=<class 'float'>
Code=BHK   khal=Boroi Hati Khal
n=0.03 type=<class 'float'>
Q=2.510450859088205 type=<class 'float'>
Code=RKK   khal=Rakhila Khal
n=0.03 type=<class 'float'>
Q=3.7549199123274764 type=<class 'float'>
Code=GZK   khal=Gazir Khal
n=0.03 type=<class 'float'>
Q=2.510450859088205 type=<class 'float'>
Code=SSK   khal=Silna -Shir Khal
n=0.03 type=<class 'float'>
Q=3.7549199123274764 type=<class 'float'>
Code=GBK   khal=Gabir Beel Khal
n=0.03 type=<class 'float'>
Q=3.7549199123274764 type=<class 'float'>
Code=KPK   khal=Khanar Par Khal
n=0.03 type=<class 'float'>
Q=3.7549199123274764 type=<class 'float'>
Code=DLB   khal=Dola Beel
n=0.03 type=<class 'float'>
Q=2.510450859088205 type=<class 'float'>
Code=DLK   khal=Dhulor

In [25]:
#driver code for Trpozoidal Section Solution
Q=10.0
B=3.0
m=2.0
s_0=0.00005
n=0.03
mysection=Trapozoidal_Xsec(Q,B,m,s_0,n)
mysection.display()
h=200
myvalues=mysection.calculate_section_params(h)
#print(myvalues)
solution_df=mysection.solve_for_h(h)
solution_df

n=0.03 type=<class 'float'>
Q=10.0 type=<class 'float'>


,trial_no,h,Q_design,n,s_0,B,m,A,P,R,T,dp_dh,R_2_3,R_5_3,f_h,f_prime_h,h_calc,Q_calc
0,1.0,200.000000,10.0,0.03,0.00005,3.0,2.0,80600.000000,897.427191,89.812300,803.000000,4.472136,20.055056,1801.190702,1.616395e+06,24159.379649,133.094512,380997.975324
1,2.0,133.094512,10.0,0.03,0.00005,3.0,2.0,35827.581780,598.216752,59.890636,535.378048,4.472136,15.307559,916.779461,5.483904e+05,12295.293601,88.492860,129266.858205
2,3.0,88.492860,10.0,0.03,0.00005,3.0,2.0,15927.451139,398.752101,39.943241,356.971440,4.472136,11.685004,466.736926,1.860699e+05,6258.523687,58.762222,43867.096853
3,4.0,58.762222,10.0,0.03,0.00005,3.0,2.0,7082.284094,265.792645,26.645899,238.048887,4.472136,8.921138,237.711746,6.313961e+04,3186.746214,38.949032,14892.148139
4,5.0,38.949032,10.0,0.03,0.00005,3.0,2.0,3150.901307,177.185367,17.783079,158.796129,4.472136,6.812993,121.155996,2.142464e+04,1623.703055,25.754105,5059.836841
5,6.0,25.754105,10.0,0.03,0.00005,3.0,2.0,1403.810181,118.175860,11.878993,106.016421,4.472136,5.206187,61.844255,7.266072e+03,828.517204,16.984134,1722.629500
6,7.0,16.984134,10.0,0.03,0.00005,3.0,2.0,627.874051,78.955358,7.952266,70.936538,4.472136,3.984073,31.682410,2.459070e+03,424.304387,11.188603,589.608269
7,8.0,11.188603,10.0,0.03,0.00005,3.0,2.0,283.935461,53.036952,5.353540,47.754410,4.472136,3.060277,16.383316,8.264947e+02,219.410133,7.421709,204.806674
8,9.0,7.421709,10.0,0.03,0.00005,3.0,2.0,132.428640,36.190889,3.659171,32.686834,4.472136,2.374589,8.689026,2.720372e+02,116.476301,5.086150,74.119778
9,10.0,5.086150,10.0,0.03,0.00005,3.0,2.0,66.996301,25.745956,2.602207,23.344601,4.472136,1.891884,4.923074,8.432283e+01,66.181937,3.812044,29.875083


In [26]:
dict2={"B":2,"C":3}
dict1={"A":1}
dict1.update(dict2)
print(dict1)

{'A': 1, 'B': 2, 'C': 3}
